In [1]:
import numpy as np
import tensorflow as tf
import keras
import keras.layers as layers

import Spartan.models as models

In [37]:

def slr_s1_model(height=176, width=176, depth=48, points_num=2, batch_size=2):
    """
    This is a simplified slr model used to debug the original one.
    """
    inputs = keras.Input((height, width, depth, 1))
    # e.x. batches*170*170*30*4*3, 4 type of coordinates, 3 dimensions
    # base_coordinate_xyz = keras.Input((width, height, depth, 4, 3))

    base_cor_rcs = models.coordinate_3d(batch_size, points_num, height, width, depth)

    # x = layers.BatchNormalization()(inputs)
    # x = layers.ReLU()(x)
    #
    # # Stage 1
    # violet_x = models.residual_block(x, downsample=True, filters=4)
    #
    # x = models.residual_block(violet_x, downsample=False, filters=4)
    # violet_x = layers.Add()([x, violet_x])
    #
    # x = models.residual_block(violet_x, downsample=False, filters=4)
    # grey_x_s1 = layers.UpSampling3D(size=2)(x)

    # heatmap_s1 = models.residual_block(grey_x_s1, downsample=False, filters=points_num)
    heatmap_s1 = inputs

    # add dropout?
    # heatmap_s1 = layers.Dropout(0.2)(heatmap_s1)

    pro_matrix_s1 = layers.Reshape((width, height, depth, points_num, 3)) \
        (tf.repeat(layers.Softmax(axis=[1, 2, 3], name="stage1_softmax")(heatmap_s1), repeats=3, axis=-1))
    outputs_s1 = tf.math.reduce_sum(layers.multiply([base_cor_rcs, pro_matrix_s1]), axis=[1, 2, 3])

    model_s1 = keras.Model(inputs, [outputs_s1, pro_matrix_s1], name="slr_stage1")


    return model_s1

In [38]:
model = slr_s1_model(4, 4, 2, 1, 2)
model.summary()

Model: "slr_stage1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 4, 4, 2, 1)]      0         
                                                                 
 stage1_softmax (Softmax)    (None, 4, 4, 2, 1)        0         
                                                                 
 tf.repeat_6 (TFOpLambda)    (None, 4, 4, 2, 3)        0         
                                                                 
 reshape_6 (Reshape)         (None, 4, 4, 2, 1, 3)     0         
                                                                 
 multiply_5 (Multiply)       (2, 4, 4, 2, 1, 3)        0         
                                                                 
 tf.math.reduce_sum_5 (TFOpL  (2, 1, 3)                0         
 ambda)                                                          
                                                        

In [25]:
batch_size = 2
points_num = 1
height = 4
width = 4
depth = 2

base_cor_rcs = models.coordinate_3d(batch_size, points_num, height, width, depth)
print(base_cor_rcs[0, :,:,0, 0, 2])
print(base_cor_rcs[0, :,:,1, 0, 2])
# print(base_cor_rcs[0, :,:,1, 0, :])

[[-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]]
[[0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5]]


In [75]:
data_bitch = np.ones((batch_size, height, width, depth, 1)) * -1
data_bitch[0, 0, 0, 1, 0] = 10
data_bitch[1, 0, 0, 1, 0] = 10

In [76]:
print(data_bitch[0, :, :, 0, 0])
print(data_bitch[0, :, :, 1, 0])

[[-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]]
[[10. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]]


In [77]:
[output, p] = model(data_bitch)
print(output)

tf.Tensor(
[[[-0.7495992 -0.7495992  0.4997328]]

 [[-0.7495992 -0.7495992  0.4997328]]], shape=(2, 1, 3), dtype=float32)


In [74]:
print(p[0, :, :, 0, 0, 0])
print(p[0, :, :, 1, 0, 0])

tf.Tensor(
[[1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]], shape=(4, 4), dtype=float32)
tf.Tensor(
[[1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 1.6693055e-05]
 [1.6693055e-05 1.6693055e-05 1.6693055e-05 9.9948227e-01]], shape=(4, 4), dtype=float32)
